In [ ]:
%pip install omegaconf==2.0.6

In [1]:
%cd FeatureExtraction
%cd video_features

[WinError 2] The system cannot find the file specified: 'FeatureExtraction'
e:\Temporal-Action-Video-Segmentation\FeatureExtraction
e:\Temporal-Action-Video-Segmentation\FeatureExtraction\video_features


In [2]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.get_device_name(0)

In [ ]:
!pip install torch
!pip install torchvision
!pip install scipy


In [35]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Select the feature type
feature_type = 'i3d'

# Load and patch the config (assuming you have the correct setup)
args = OmegaConf.load(build_cfg_path(feature_type))
args.device = device
args.flow_type = 'raft'

# Load the model (assuming you have the correct setup)
extractor = ExtractI3D(args)

# Directory containing video files
video_dir = '../../Input/data'  # Change this to your video directory

# Loop through subdirectories in the video directory
for sub_dir in os.listdir(video_dir):
    sub_dir_path = os.path.join(video_dir, sub_dir)

    if os.path.isdir(sub_dir_path):
        # Create a corresponding subdirectory in the extracted features folder
        output_sub_dir = os.path.join('../../Output/features')
        os.makedirs(output_sub_dir, exist_ok=True)

        # Create a folder for ground truth if it doesn't exist
        ground_truth_dir = os.path.join('../../Output/groundTruth')
        os.makedirs(ground_truth_dir, exist_ok=True)

        # Extract features for all video files in the subdirectory
        for video_file in os.listdir(sub_dir_path):
            if video_file.endswith('.mp4') or video_file.endswith('.avi'):
                video_name = os.path.splitext(video_file)[0]
                npy_filename = os.path.join(output_sub_dir, f'{video_name}_flow.npy')
                npy_filename2 = os.path.join(output_sub_dir, f'{video_name}_fps.npy')
                npy_filename3 = os.path.join(output_sub_dir, f'{video_name}_timestamps_ms.npy')
                npy_filename4 = os.path.join(output_sub_dir, f'{video_name}_rgb.npy')
                txt_filename = os.path.join(ground_truth_dir, f'{video_name}_flow.txt')
                txt_filename2 = os.path.join(ground_truth_dir, f'{video_name}_fps.txt')
                txt_filename3 = os.path.join(ground_truth_dir, f'{video_name}_timestamps_ms.txt')
                txt_filename4 = os.path.join(ground_truth_dir, f'{video_name}_rgb.txt')
                print(f'Check {txt_filename}')
                # Skip if the files for this video have already been processed
                if os.path.exists(txt_filename) and os.path.exists(txt_filename2) and os.path.exists(txt_filename3) and os.path.exists(txt_filename4) and os.path.exists(npy_filename) and os.path.exists(npy_filename2) and os.path.exists(npy_filename3) and os.path.exists(npy_filename4):
                    print(f'Skipped {video_file} (Already processed)')
                    continue

                    
                video_path = os.path.join(sub_dir_path, video_file)
                print(f'Extracting features for {video_file}')

                cap = cv2.VideoCapture(video_path)
                fps = int(cap.get(cv2.CAP_PROP_FPS))
                num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                cap.release()
                if not (os.path.exists(npy_filename) or os.path.exists(npy_filename2) or os.path.exists(npy_filename3) or os.path.exists(npy_filename4)):
                  feature_dict = extractor.extract(video_path)
                  for key, value in feature_dict.items():

                      reshaped_feature = np.resize(value, (2048, num_frames))
                      # Create a separate .npy file for each feature type
                      npy_filename = os.path.join(output_sub_dir, f'{video_file.split(".")[0]}_{key}.npy')
                      txt_filename = os.path.join(ground_truth_dir, f'{video_file.split(".")[0]}_{key}.txt')
                      np.save(npy_filename, reshaped_feature)

                      with open(txt_filename, 'w') as txt_file:
                              for _ in range(num_frames):
                                  txt_file.write(os.path.splitext(video_file)[0].split('_')[0] + '\n')  # Write sub_dir_path `num_frames` times
                      print(f'Saved {key} feature as {npy_filename}')
                elif not (os.path.exists(txt_filename) or os.path.exists(txt_filename2) or os.path.exists(txt_filename3) or os.path.exists(txt_filename4)):

                  for key in ["flow","fps","timestamps_ms", "rgb"]:

                      txt_filename = os.path.join(ground_truth_dir, f'{video_file.split(".")[0]}_{key}.txt')

                      with open(txt_filename, 'w') as txt_file:
                              for _ in range(num_frames):
                                  txt_file.write(os.path.splitext(video_file)[0].split('_')[0] + '\n')  # Write sub_dir_path `num_frames` times
                      print(f'Saved {key} feature as {txt_filename}')



                print(f'Processed {video_file}')


Check ../../Output/groundTruth\Biking_g01_c01_flow.txt
Extracting features for Biking_g01_c01.avi
Saved flow feature as ../../Output/groundTruth\Biking_g01_c01_flow.txt
Saved fps feature as ../../Output/groundTruth\Biking_g01_c01_fps.txt
Saved timestamps_ms feature as ../../Output/groundTruth\Biking_g01_c01_timestamps_ms.txt
Saved rgb feature as ../../Output/groundTruth\Biking_g01_c01_rgb.txt
Processed Biking_g01_c01.avi
Check ../../Output/groundTruth\Biking_g01_c02_flow.txt
Extracting features for Biking_g01_c02.avi
Saved flow feature as ../../Output/groundTruth\Biking_g01_c02_flow.txt
Saved fps feature as ../../Output/groundTruth\Biking_g01_c02_fps.txt
Saved timestamps_ms feature as ../../Output/groundTruth\Biking_g01_c02_timestamps_ms.txt
Saved rgb feature as ../../Output/groundTruth\Biking_g01_c02_rgb.txt
Processed Biking_g01_c02.avi
Check ../../Output/groundTruth\Biking_g01_c03_flow.txt
Extracting features for Biking_g01_c03.avi
Saved flow feature as ../../Output/groundTruth\Bikin

In [36]:
import os
import random
from collections import defaultdict

# Define input and output directories
input_dir = '../../Output/groundTruth'
splits_dir = '../../Output/splits'

# Function to create bundles for training and testing based on file names
def create_bundles(input_dir, splits_dir):
    # Dictionary to store files based on categories
    categories = defaultdict(list)
    
    # Iterate through files in the input directory and categorize them based on names
    for filename in os.listdir(input_dir):
        # Get category name from the file name (up to the first underscore)
        category = filename.split('_')[0]
        # Categorize files based on their names
        if filename.endswith('_rgb.txt'):
            categories['rgb'].append(filename)
        elif filename.endswith('_timestamps_ms.txt'):
            categories['timestamps'].append(filename)
        elif filename.endswith('_flow.txt'):
            categories['flow'].append(filename)
        elif filename.endswith('_fps.txt'):
            categories['fps'].append(filename)
    
    # Create bundles for testing and training based on file categories
    for idx, (category, files) in enumerate(categories.items(), start=1):
        # Calculate the number of files for training and testing (80% for training, 20% for testing)
        num_train_files = int(0.8 * len(files))
        train_files = random.sample(files, num_train_files)
        test_files = list(set(files) - set(train_files))
        
        # Create bundle files for testing
        with open(os.path.join(splits_dir, f'test.split{idx}.bundle'), 'a') as test_bundle:
            for file in test_files:
                test_bundle.write(f"{file}\n")
        
        # Create bundle files for training
        with open(os.path.join(splits_dir, f'train.split{idx}.bundle'), 'a') as train_bundle:
            for file in train_files:
                train_bundle.write(f"{file}\n")

# Create splits directory if it doesn't exist
if not os.path.exists(splits_dir):
    os.makedirs(splits_dir)

# Call the function to create bundles
create_bundles(input_dir, splits_dir)

print("Bundles created successfully.")


Bundles created successfully.


In [34]:
import os

# Directory containing files with "v_" prefix
input_dir = '../../Input/data/ucfWalkingwithdog'

# Loop through files in the input directory
for filename in os.listdir(input_dir):
    # Check if the filename starts with "v_"
    if filename.startswith('v_'):
        # Remove the "v_" prefix from the filename
        new_filename = filename[2:]
        
        # Rename the file
        os.rename(os.path.join(input_dir, filename), os.path.join(input_dir, new_filename))
        
        print(f'Renamed {filename} to {new_filename}')


Renamed v_WalkingWithDog_g01_c01.avi to WalkingWithDog_g01_c01.avi
Renamed v_WalkingWithDog_g01_c02.avi to WalkingWithDog_g01_c02.avi
Renamed v_WalkingWithDog_g01_c03.avi to WalkingWithDog_g01_c03.avi
Renamed v_WalkingWithDog_g01_c04.avi to WalkingWithDog_g01_c04.avi
Renamed v_WalkingWithDog_g02_c01.avi to WalkingWithDog_g02_c01.avi
Renamed v_WalkingWithDog_g02_c02.avi to WalkingWithDog_g02_c02.avi
Renamed v_WalkingWithDog_g02_c03.avi to WalkingWithDog_g02_c03.avi
Renamed v_WalkingWithDog_g02_c04.avi to WalkingWithDog_g02_c04.avi
Renamed v_WalkingWithDog_g02_c05.avi to WalkingWithDog_g02_c05.avi
Renamed v_WalkingWithDog_g02_c06.avi to WalkingWithDog_g02_c06.avi
Renamed v_WalkingWithDog_g03_c01.avi to WalkingWithDog_g03_c01.avi
Renamed v_WalkingWithDog_g03_c02.avi to WalkingWithDog_g03_c02.avi
Renamed v_WalkingWithDog_g03_c03.avi to WalkingWithDog_g03_c03.avi
Renamed v_WalkingWithDog_g03_c04.avi to WalkingWithDog_g03_c04.avi
Renamed v_WalkingWithDog_g03_c05.avi to WalkingWithDog_g03_c05

In [37]:
# Directory containing ground truth files
ground_truth_dir = '../../Output/groundTruth'  # Change this to your ground truth directory

# Create a folder for splits if it doesn't exist
output_dir = '../../Output'
os.makedirs(output_dir, exist_ok=True)

# List all ground truth files
ground_truth_files = os.listdir(ground_truth_dir)

# Create a dictionary to store category indices
category_indices = {}

# Extract the category names and assign indices
category_index = 0
for filename in ground_truth_files:
    parts = filename.split('_')
    category =parts[0]  # Extract the category name
    if category not in category_indices:
        category_indices[category] = category_index
        category_index += 1

# Create the mapping content
mapping_content = '\n'.join([f'{index} {category}' for category, index in category_indices.items()])

# Write the mapping content to the mapping.txt file
mapping_filename = os.path.join(output_dir, 'mapping.txt')
with open(mapping_filename, 'w') as mapping_file:
    mapping_file.write(mapping_content+'\n')

print(f'Created {mapping_filename} successfully.')


Created ../../Output\mapping.txt successfully.


In [ ]:
!pip freeze